In [59]:
import pandas as pd
import matplotlib.pyplot as plt

import data

In [60]:
all_ptraces = pd.read_csv(data.DIR_PATH / 'bitter_coils_ptraces_hres', skiprows=7, delim_whitespace=True)
all_ptraces


/tmp/ipykernel_12160/431026227.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  all_ptraces = pd.read_csv(data.DIR_PATH / 'bitter_coils_ptraces_hres', skiprows=7, delim_whitespace=True)


,x,y,z,Particle,Time,x-velocity,y-velocity,z-velocity,Particle.1,velocity,magnitude
0,-0.004763,-0.032274,-6.000000e-15,0,0.000,2.979609e-04,0.240303,1.658947e-04,0.240303,NaN,NaN
1,-0.004762,-0.031060,1.438445e-07,0,0.005,-9.771369e-06,0.247167,-6.499440e-07,0.247776,NaN,NaN
2,-0.004761,-0.029748,6.197992e-07,0,0.010,1.389004e-03,0.286346,7.817751e-04,0.289853,NaN,NaN
3,-0.004763,-0.029535,3.784024e-07,0,10.000,-4.740000e-03,0.311119,-1.197402e-03,0.311506,NaN,NaN
4,-0.004763,-0.032274,1.270000e-05,1,0.000,-1.854751e-12,0.240418,0.000000e+00,0.240418,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
270883,0.004762,-0.029830,2.919780e-04,623,10.000,4.464234e-04,0.177894,1.389228e-04,0.177927,NaN,NaN
270884,0.004763,-0.032274,3.048000e-04,624,0.000,-2.160056e-04,0.172294,-3.034582e-04,0.172295,NaN,NaN
270885,0.004762,-0.031411,3.045665e-04,624,0.005,-4.557582e-06,0.172671,-4.282985e-05,0.172497,NaN,NaN
270886,0.004762,-0.030548,3.045500e-04,624,0.010,6.922993e-05,0.173183,2.783239e-05,0.173170,NaN,NaN


In [61]:
particle_traces = {particle: data for particle, data in all_ptraces.groupby('Particle')}
particle_traces

{0:           x         y             z  Particle    Time  x-velocity  y-velocity  \
 0 -0.004763 -0.032274 -6.000000e-15         0   0.000    0.000298    0.240303   
 1 -0.004762 -0.031060  1.438445e-07         0   0.005   -0.000010    0.247167   
 2 -0.004761 -0.029748  6.197992e-07         0   0.010    0.001389    0.286346   
 3 -0.004763 -0.029535  3.784024e-07         0  10.000   -0.004740    0.311119   
 
      z-velocity  Particle.1  velocity  magnitude  
 0  1.658947e-04    0.240303       NaN        NaN  
 1 -6.499440e-07    0.247776       NaN        NaN  
 2  7.817751e-04    0.289853       NaN        NaN  
 3 -1.197402e-03    0.311506       NaN        NaN  ,
 1:           x         y         z  Particle    Time    x-velocity  y-velocity  \
 4 -0.004763 -0.032274  0.000013         1   0.000 -1.854751e-12    0.240418   
 5 -0.004762 -0.031060  0.000013         1   0.005  4.410815e-05    0.247139   
 6 -0.004762 -0.029747  0.000013         1   0.010 -1.596389e-03    0.287250   
 

In [62]:
# Also need to check if a particle is frozen. Basically, we need only consider the particles that have landed on the surface given by the coordinates:

# (4.762500000, -32.274010034, 2.286000000)
# (-4.762500000, -32.274010034, 2.286000000)
# (-4.762500000, -32.274010034 , 2.895600000 )
# (4.762500000 , -32.274010034, 2.895600000 )

# These are in mm. Notice the y coordinate is fixed, so we need to check if the particle's last point is within the x and z bounds of these coordinates, and that the y-coordinate is within some tolerance of the y-coordinate of the surface. 

ytol = 0.5 # mm
z_bounds = (2.286, 2.896) # mm
x_bounds = (-4.763, 4.763) # mm

# Bounds in m
ytol /= 1000
z_bounds = (z_bounds[0] / 1000, z_bounds[1] / 1000) # m
x_bounds = (x_bounds[0] / 1000, x_bounds[1] / 1000) # m
# Check if the particle is frozen   
def is_frozen(ptrace):
    last_point = ptrace.iloc[-1]
    x, y, z = last_point['x'], last_point['y'], last_point['z']
    return not ((x_bounds[0] <= x <= x_bounds[1]) and (z_bounds[0] <= z <= z_bounds[1]) and (y - ytol <= y <= y + ytol))






In [66]:

# For each particle trace, reset the index
# non_frozen_ptraces = particle_traces.copy()

for particle_num, ptrace in particle_traces.items():
    ptrace.reset_index(drop=True, inplace=True)
    
    # Only keep the x, y, z columns
    ptrace = ptrace[['x', 'y', 'z']]
    
    particle_traces[particle_num] = ptrace
    
    # Check for duplicate entries with respect to the x, y, z coordinates
    # Then trim those duplicates off (keep the first occurrence)
    # duplicates = ptrace.duplicated(subset=['x', 'y', 'z'], keep='first')
    duplicates = ptrace[ptrace.duplicated(subset=['x', 'y', 'z'], keep=False)]
    print(f"Particle {particle_num} has {len(duplicates)} duplicates.")
    ptrace = ptrace.drop_duplicates(subset=['x', 'y', 'z'], keep='first')
    
    # Check if the particle is frozen
    if is_frozen(ptrace):
        print(f"Particle {particle_num} is frozen. Skipping...")
        continue
    
    
    # Save the dataframe to a CSV file with the naming convention: particle_01.csv and so on
    ptrace.to_csv(data.DIR_PATH /'coil_data_trimmed'/ f'particle_{particle_num:02d}.csv', index=False)
    
    




Particle 0 has 0 duplicates.
Particle 0 is frozen. Skipping...
Particle 1 has 0 duplicates.
Particle 1 is frozen. Skipping...
Particle 2 has 0 duplicates.
Particle 2 is frozen. Skipping...
Particle 3 has 0 duplicates.
Particle 3 is frozen. Skipping...
Particle 4 has 0 duplicates.
Particle 4 is frozen. Skipping...
Particle 5 has 0 duplicates.
Particle 5 is frozen. Skipping...
Particle 6 has 0 duplicates.
Particle 6 is frozen. Skipping...
Particle 7 has 0 duplicates.
Particle 7 is frozen. Skipping...
Particle 8 has 0 duplicates.
Particle 8 is frozen. Skipping...
Particle 9 has 0 duplicates.
Particle 9 is frozen. Skipping...
Particle 10 has 0 duplicates.
Particle 10 is frozen. Skipping...
Particle 11 has 0 duplicates.
Particle 11 is frozen. Skipping...
Particle 12 has 0 duplicates.
Particle 12 is frozen. Skipping...
Particle 13 has 0 duplicates.
Particle 13 is frozen. Skipping...
Particle 14 has 0 duplicates.
Particle 14 is frozen. Skipping...
Particle 15 has 0 duplicates.
Particle 15 is 

In [ ]:
particle_traces[28]


,x,y,z
0,-0.004366,-0.032274,0.000038
1,-0.004377,-0.031064,0.000038
2,-0.004429,-0.029765,0.000038
3,-0.004867,-0.028240,0.000038
4,-0.005838,-0.027101,0.000037
...,...,...,...
963,0.004617,-0.029850,0.002343
964,0.004604,-0.030505,0.002343
965,0.004597,-0.031122,0.002343
966,0.004594,-0.031720,0.002343
